In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers
!pip install torch
!pip install numpy
!pip install pronouncing
!pip install frozendict
!pip install sacremoses
!pip install Phyme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 35.2 MB/s 
     |████████████████████████████████| 6.6 MB 2.2 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
     |████████████████████████████████| 596 kB 38.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 939 kB 35.8 MB/s 
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6252 sha256=083acbdf3fc1bccf7ffa8fb99376274b11495ecb50481bd0e71ea7c3c919930e
  S

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%cd drive/MyDrive/eth_courses/Autumn22/nlp/spanningtrees/

/content/drive/MyDrive/eth_courses/Autumn22/nlp/spanningtrees


In [3]:
from poem_generation import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
100%|██████████| 267735/267735 [00:00<00:00, 298356.75it/s]


In [4]:
model = model.to(torch.device('cuda'))
device = torch.device('cuda')

In [5]:
lcp = LineCommaPolice([6])
poem_processor = syllable_line_rhyming_logits_smarter(num_syllables=[6], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Standing here all lonely,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 6]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 1
Standing here all lonely, next
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('next', 1)]
5
num_syllables: 2
Standing here all lonely, within
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('within', 2)]
6
num_syllables: 2
Standing here all lonely, next all
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('next', 1), ('all', 1)]
6
num_syllables: 0
Standing here all lonely, within progressively
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('within', 2), ('progressively', 4)]
4
num_syllables: 0
Standing here all lonely, next all physically
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('next', 1), ('all', 1), ('physically', 4)]
4
num_syllables

In [ ]:
aaa

In [55]:
def get_last_word_indicators(word_syllables, rep):
    # rep: e.g. [7] for all lines 7 syllables, or [3,5] for alternating 3, 5
    # syllable lines.
    # e.g. [2, 0, 4, 4, 1, 0, 1, 1, 5, 0, 1] as input gives
    # [0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3]. so 3rd word is the last of
    # its line and so on.

    if type(rep) is int:
        rep = [rep]

    i = 0
    line_num = 0
    current_line_num_syllables = 0
    last_word_indicator = []
    while i < len(word_syllables):
        syllable_target = rep[line_num % len(rep)]
        current_line_num_syllables += word_syllables[i]
        print(current_line_num_syllables)
        if current_line_num_syllables >= syllable_target:
            line_num += 1
            current_line_num_syllables -= syllable_target
        print(line_num)
        last_word_indicator.append(line_num)
        i += 1

    # e.g. 0, 0, 1, 1, ... means the 3rd word is the first to add up to n_syllables. I.e. it's the last word of the line

    # 0, 0, 1, 1, .... - 0, 0, 0, 1, 1, ... = 0, 0, 1, 0, 0, ... indicators where that last word is. Each 1 is the last
    # word of the line (potentially followed by some punctuation/ other zero syllable words still on the same line which
    # don't count.
    last_word_indicator = np.concatenate([last_word_indicator, np.zeros(1)]) - np.concatenate(
        [np.zeros(1), last_word_indicator])
    
    return line_num, current_line_num_syllables, last_word_indicator, syllable_target


text = "Standing here all lonely, upright"
word_syllables = get_text_syllables(text)
syllables = [sylls for word, sylls in word_syllables]
get_last_word_indicators(syllables, [6])
syllables

2
0
3
0
4
0
6
1
0
1
2
1


(1, 2, array([ 0.,  0.,  0.,  1.,  0.,  0., -1.]), 6)

[2, 1, 1, 2, 0, 2]

In [59]:
def get_last_word_indicators(word_syllables, rep):
    # rep: e.g. [7] for all lines 7 syllables, or [3,5] for alternating 3, 5
    # syllable lines.
    # e.g. [2, 0, 4, 4, 1, 0, 1, 1, 5, 0, 1] as input gives
    # [0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3]. so 3rd word is the last of
    # its line and so on.

    if type(rep) is int:
        rep = [rep]

    i = 0
    line_num = 0
    current_line_num_syllables = 0
    last_word_indicator = []
    while i < len(word_syllables):
        syllable_target = rep[line_num % len(rep)]
        current_line_num_syllables += word_syllables[i]
        if current_line_num_syllables >= syllable_target:
            line_num += 1
            current_line_num_syllables -= syllable_target
        last_word_indicator.append(line_num)
        i += 1

    # e.g. 0, 0, 1, 1, ... means the 3rd word is the first to add up to n_syllables. I.e. it's the last word of the line

    # 0, 0, 1, 1, .... - 0, 0, 0, 1, 1, ... = 0, 0, 1, 0, 0, ... indicators where that last word is. Each 1 is the last
    # word of the line (potentially followed by some punctuation/ other zero syllable words still on the same line which
    # don't count.
    last_word_indicator = np.concatenate([last_word_indicator, np.zeros(1)]) - np.concatenate(
        [np.zeros(1), last_word_indicator])
    
    return  current_line_num_syllables, line_num, last_word_indicator, syllable_target

text = "Standing here all lonely, upright"
word_syllables = get_text_syllables(text)
syllables = [sylls for word, sylls in word_syllables]
get_last_word_indicators(syllables, [6])

text = "Standing here all lonely"
word_syllables = get_text_syllables(text)
syllables = [sylls for word, sylls in word_syllables]
get_last_word_indicators(syllables, [6])


text = "Standing here all lonely, she"
word_syllables = get_text_syllables(text)
syllables = [sylls for word, sylls in word_syllables]
get_last_word_indicators(syllables, [6])


text = "Standing here all lonely, they went together to,"
word_syllables = get_text_syllables(text)
syllables = [sylls for word, sylls in word_syllables]
get_last_word_indicators(syllables, [6])

class syllable_line_rhyming_logits_smarter(syllable_line_logits):
    def __init__(self, num_syllables=[3,5], sd=3.5):
        super().__init__(num_syllables=num_syllables)
        self.sd = sd

    def __call__(self, input_ids, scores):

        banned_tokens = []
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            text = tokenizer.decode(beam_input_ids)
            word_syllables = get_text_syllables(text)

            # how many syllables we're into the current line.
            current_line_num_syllables, actual_line_num, last_word_indicator, syllable_target = \
                get_last_word_indicators([sylls for word, sylls in word_syllables], self.num_syllables)


            last_rhyme_word_index = np.where(last_word_indicator == 1)[0]
            if len(last_rhyme_word_index) > 0:
                last_rhyme_word_index = last_rhyme_word_index[-1]
            else:
                last_rhyme_word_index = None

            if last_rhyme_word_index:
                last_rhyme_word = word_syllables[last_rhyme_word_index][0]
                rhyming_words = set(get_rhyming_words(last_rhyme_word))
                rhyming_words_idxs = np.array([vocab['sym2idx'][word] for word in rhyming_words if
                                               word in vocab['sym2idx']])
            else:
                rhyming_words = set([])
                rhyming_words_idxs = np.array([])

            num_syllables = current_line_num_syllables
            target_syllables = self.num_syllables[actual_line_num % len(self.num_syllables)]
            # we're at the end of a line - don't ban as all welcome: 0, 1, 2, etc. syllable words
            if num_syllables == target_syllables:
                num_syllables = 0
                target_syllables = self.num_syllables[(actual_line_num+1) % len(self.num_syllables)]

            to_ban = [idxs for i, idxs in enumerate(self.syllable_idxs) if i + 1 + num_syllables > target_syllables]

            just_fit_idxs = self.syllable_idxs[target_syllables - num_syllables - 1]
            just_fit_rhyming_intersection = np.intersect1d(rhyming_words_idxs, just_fit_idxs, assume_unique=True)
            # just_fit_rhyming_intersection = set(just_fit_rhyming_intersection)  # indices within just_fit_idxs
            just_fit_idxs_ban = np.array([idx for idx in just_fit_idxs if not idx in just_fit_rhyming_intersection])

            beam_scores[just_fit_rhyming_intersection] += self.sd

            to_ban.append(just_fit_idxs_ban)

            if to_ban:
                print(f'num_syllables: {num_syllables}')
                print(text)
                print(quick_neatify_text(text))
                print(len(to_ban))
            banned_tokens.append(np.concatenate(to_ban) if to_ban else [])

        scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
        return scores


class LineCommaPolice(LogitsProcessor):
    def __init__(self, num_syllables):
        self.num_syllables = num_syllables

    def __call__(self, input_ids, scores):
      # for every beam (partially generated sentence)
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            # get the last token of this beam
            text = tokenizer.decode(beam_input_ids)
            word_syllables = get_text_syllables(text)

            # how many syllables we're into the current line.
            current_line_num_syllables, actual_line_num, last_word_indicator, syllable_target = \
                get_last_word_indicators([sylls for word, sylls in word_syllables], self.num_syllables)

            if current_line_num_syllables == 0 and text[-1] != ',':
                beam_scores[2] += 10.0
        return scores


logits_processor = LogitsProcessorList([LineCommaPolice([3]), syllable_line_rhyming_logits_smarter([6], sd=4.2)]) #syllable_line_rhyming_logits_smarter([6], sd=4.2)])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Standing here all lonely,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=4,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 6]))
  print('##########')


(2, 1, array([ 0.,  0.,  0.,  1.,  0.,  0., -1.]), 6)

(0, 1, array([ 0.,  0.,  0.,  1., -1.]), 6)

(1, 1, array([ 0.,  0.,  0.,  1.,  0.,  0., -1.]), 6)

(0, 2, array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0., -2.]), 6)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 1
Standing here all lonely, while
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('while', 1)]
5
num_syllables: 1
Standing here all lonely, in
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('in', 1)]
5
num_syllables: 4
Standing here all lonely, accessible
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('accessible', 4)]
8
num_syllables: 2
Standing here all lonely, upper
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('upper', 2)]
6
num_syllable

In [45]:
word_syllables

[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]

In [43]:
class line_comma_police(LogitsProcessor):
    def __call__(self, input_ids, scores):
      banned_tokens = []
      # for every beam (partially generated sentence)
      for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
        # get the last token of this beam
        last_word = tokenizer.decode(beam_input_ids[-1])
        text = tokenizer.decode(beam_input_ids)
        num_words = len([x for x in text if x == ' ']) + 1
        if num_words % 3 == 0 and text[-1] not in [',', '.']:
          beam_scores[2] += 10.0
          banned_tokens.append([])
      # set the scores of all banned tokens over the beams to -inf
      scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
      return scores

class LineCommaPolice(LogitsProcessor):
    def __init__(self, num_syllables):
        self.num_syllables = num_syllables

    def __call__(self, input_ids, scores):
      # for every beam (partially generated sentence)
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            # get the last token of this beam
            text = tokenizer.decode(beam_input_ids)
            word_syllables = get_text_syllables(text)
    
            current_line_num_syllables, actual_line_num, last_word_indicator = \
                get_last_word_indicators([sylls for word, sylls in word_syllables], self.num_syllables)
    
            print(current_line_num_syllables)
            print(word_syllables)
            if current_line_num_syllables ==  and text[-1] not in [',', '.']:
                beam_scores[2] += 10.0
                # print(text)
        return scores

logits_processor = LogitsProcessorList([LineCommaPolice([6])])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Standing here all lonely,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


6
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
6
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
2
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('lonely', 2)]
1
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('where', 1)]
2
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('lonely', 2), ('â\x80\x93', 0)]
4
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('where', 1), ('loneliness', 3)]
3
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('lonely', 2), ('â\x80\x93', 0), ('camps', 1)]
6
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('where', 1), ('loneliness', 3), ('contrasts', 2)]
4
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('lonely', 2), ('â\x80\x93', 0), ('camps', 1), ('and', 1)]
2
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('where', 1), ('loneliness', 3), ('contrasts', 2

In [42]:

outputs

tensor([[11912,  1744,    73, 13750,     2,  1129,  1195,     2,   756,     1,
           549,   866,     3, 35961,  5101,     1,  2656,     4,     1, 71937,
           549,   627,    23,     1, 65703, 34888,     3,    89,     1,   214],
        [11912,  1744,    73, 13750,     2, 95080,     3,    13,   494, 48279,
             4,  4526,  3082,     1,   494,  4265,     7,  1134,     2,    34,
           168,  4879,    56, 13471,     3,    10, 16826,    12, 11546,    14]],
       device='cuda:0')

In [40]:
for output in outputs:
  print(tokenizer.decode(output))
  print('######')

Standing here all lonely, was fulfilled, and was never filmed. Manchester United's rivals Liverpool City FC and Nottingham Forest were unable to compete. This meant
######
Standing here all lonely, lonely old old part new parts newly released parts,. older parts added to the newly restored parts. [1] The old
######


In [37]:
class syllable_line_rhyming_logits_smarter(syllable_line_logits):
    def __init__(self, num_syllables=[3,5], sd=3.5):
        super().__init__(num_syllables=num_syllables)
        self.sd = sd
        self.except_comma_and_eos = np.array([1] + list(range(3, len(vocab['idx2sym']))))

    def __call__(self, input_ids, scores):

        banned_tokens = []
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            text = tokenizer.decode(beam_input_ids)
            word_syllables = get_text_syllables(text)

            # how many syllables we're into the current line.
            current_line_num_syllables, actual_line_num, last_word_indicator = \
                get_last_word_indicators([sylls for word, sylls in word_syllables], self.num_syllables)

            if current_line_num_syllables == 0 and text[-1] != ',':
                beam_scores[2] += 10.0
                banned_tokens.append([])
                continue


            last_rhyme_word_index = np.where(last_word_indicator == 1)[0]
            if len(last_rhyme_word_index) > 0:
                last_rhyme_word_index = last_rhyme_word_index[-1]
            else:
                last_rhyme_word_index = None

            if last_rhyme_word_index:
                last_rhyme_word = word_syllables[last_rhyme_word_index][0]
                rhyming_words = set(get_rhyming_words(last_rhyme_word))
                rhyming_words_idxs = np.array([vocab['sym2idx'][word] for word in rhyming_words if
                                               word in vocab['sym2idx']])
            else:
                rhyming_words = set([])
                rhyming_words_idxs = np.array([])

            num_syllables = current_line_num_syllables
            target_syllables = self.num_syllables[actual_line_num % len(self.num_syllables)]
            # we're at the end of a line - don't ban as all welcome: 0, 1, 2, etc. syllable words
            if num_syllables == target_syllables:
                num_syllables = 0
                target_syllables = self.num_syllables[(actual_line_num+1) % len(self.num_syllables)]

            to_ban = [idxs for i, idxs in enumerate(self.syllable_idxs) if i + 1 + num_syllables > target_syllables]

            just_fit_idxs = self.syllable_idxs[target_syllables - num_syllables - 1]
            just_fit_rhyming_intersection = np.intersect1d(rhyming_words_idxs, just_fit_idxs, assume_unique=True)
            # just_fit_rhyming_intersection = set(just_fit_rhyming_intersection)  # indices within just_fit_idxs
            just_fit_idxs_ban = np.array([idx for idx in just_fit_idxs if not idx in just_fit_rhyming_intersection])

            beam_scores[just_fit_rhyming_intersection] += self.sd

            to_ban.append(just_fit_idxs_ban)

            if to_ban:
                print(f'num_syllables: {num_syllables}')
                print(text)
                print(quick_neatify_text(text))
                print(len(to_ban))
            banned_tokens.append(np.concatenate(to_ban) if to_ban else [])

        scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
        return scores


class BanNonWords(LogitsProcessor):
    def __init__(self):
        # We will allow the first three: <eos> , .
        self.vocab_invalid_idx = [idx for idx, sym in enumerate(vocab['idx2sym']) if not sym in cmu][2:]

    def __call__(self, input_ids, scores):
        banned_tokens = []
        # for every beam (partially generated sentence)
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            banned_tokens.append(self.vocab_invalid_idx)
        # set the scores of all banned tokens over the beams to -inf
        scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
        return scores

poem_processor = syllable_line_rhyming_logits_smarter(num_syllables=[6, 6], sd = 4.2)
word_banning_processor = BanNonWords()

# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Standing here all lonely,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 6]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 3
Standing here all lonely, uneasy
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('uneasy', 3)]
7
num_syllables: 1
Standing here all lonely, with
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('with', 1)]
5
num_syllables: 3
Standing here all lonely, uneasy,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('uneasy', 3), (',', 0)]
7
num_syllables: 1
Standing here all lonely, with,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('with', 1), (',', 0)]
5
num_syllables: 5
Standing here all lonely, uneasy, confused
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('uneasy', 3), (',', 0), ('confused', 2)]
9
num_syllables: 0
Standing here all lonely, with,

In [38]:
for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 6]))
  print('##########')

 Standing here all lonely ,
uneasy , confused ,
##########
 Standing here all lonely ,
with , interchangeably
intermixed with many
of others , possibly
breaking the industry
by temporarily
following misery
and insecurity
and other seemingly
being physically
placed
##########


In [32]:
for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6,7]))
  print('##########')

 Standing here all lonely ,
isolated just really
broke was standard daily
by hosting ordinary
crowds at clubs in many
cities in a hostile , ,
##########
 Standing here all lonely ,
and with adequate money ,
##########
 Standing here all lonely ,
dreamy , and melancholy
sympathetically
beautiful history ,
##########


In [15]:
for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6,7,7, 6]))
  print('##########')

 Standing here all lonely ,
wrapped in my own misery ,
I thought upon the only
man I could possibly
go home . officially
I 'm getting somebody
else out of community .
Would I actually
love you ? ... everybody
thinks [ that ] , what would I do
##########
 Standing here all lonely ,
wrapped in my own misery ,
I thought upon the only
honorable entry . ``
Yes , '' he wrote back . early
on in the morning , vaguely
in his mouth , he finally
found out that she was 13 , 18 , 20
##########
 Standing here all lonely ,
wrapped in my own misery ,
I thought upon the only
thing about which I 'd 'couldn't't ' . FrÃ©dÃ©ric Passy , , 15 , , 10 , Theorie algebraischen 2 , 1 , 2.1. , IX
##########


In [22]:
logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([syllable_line_logits(num_syllables=6)])
logits_processor = LogitsProcessorList([syllable_line_rhyming_logits(num_syllables=7)])
logits_processor = LogitsProcessorList([syllable_line_rhyming_logits_smarter(num_syllables=[6,6,7,7,8,8])])
logits_processor = LogitsProcessorList([syllable_line_rhyming_logits_smarter(num_syllables=[7], sd = 4.2)])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Swiftly through the trees she ran,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=3,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=50,
    num_beams=7,
    do_sample=True
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6,7,7, 6]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she',

In [ ]:
for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6,7,7, 6]))
  print('##########')

In [21]:
text = tokenizer.decode(outputs[2], skip_special_tokens=True)
print(text)

Standing here all lonely, lonely and lonely lonely.


In [ ]:
import numpy as np
syllable_words = []
idx_to_num_syllables = []
for i in range(15):
  syllable_words.append([])
for word in tqdm.tqdm(vocab['idx2sym'], total=len(vocab['idx2sym'])):
  ns = count_syllables_cmu(word)
  syllable_words[ns].append(word)
  idx_to_num_syllables.append(ns)

idx_to_num_syllables = np.array(idx_to_num_syllables)

100%|██████████| 267735/267735 [00:00<00:00, 282864.67it/s]


In [ ]:
# tokenizerxl = AutoTokenizer.from_pretrained('transfoxl')
# modelxl = AutoModelForCausalLM.from_pretrained("transfoxl")
import transformers
import sacremoses as sm

from transformers import TransfoXLTokenizer, TransfoXLModel, TransfoXLLMHeadModel
import torch

tokenizer = TransfoXLTokenizer.from_pretrained("transfo-xl-wt103")
model = TransfoXLLMHeadModel.from_pretrained("transfo-xl-wt103")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizergpt = AutoTokenizer.from_pretrained('gpt2')

{'input_ids': [6409]}

In [ ]:
from transformers import (
        BeamSearchScorer,
        LogitsProcessorList,
        StoppingCriteriaList,
        MaxLengthCriteria
    )
import torch

# how many beams to track during the Viterbi algorithm
num_beams = 5
# how many beams to return after the algorithm
num_return_beams = 5

# the prompt to continue
prompt = 'My friend Jim'

# tokenizing the prompt
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids']

# instantiating a BeamSearchScorer
beam_scorer = BeamSearchScorer(
    batch_size=prompt_tokenized.shape[0],
    num_beams=num_beams,
    num_beam_hyps_to_keep=num_return_beams,
    device=model.device
)

# instantiating a list of LogitsProcessor instances
# using our custom ABCLogits class
# logits_processor = LogitsProcessorList([ABCLogits(tokenizer.vocab)])
# logits_processor = LogitsProcessorList([n_word_lines2(n_syllables=7)])
logits_processor = LogitsProcessorList([])

# # printing the output beams
# for index, output_tokenized in enumerate(generated):
#   output = tokenizer.decode(output_tokenized)
#   print(f'beam {index}: {output}')

outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
for output in outputs:
  print(tokenizer.decode(output, skip_special_tokens=True))
  print('############')

Strong son of god, immortal love,
 who weeps with many bones.


Let us battle these merciless demons!

(radio: Cros
############
Strong son of god, immortal love,
 who weeps to death in glad tears, that his sprightliness hath been assailed, he have
############
Strong son of god, immortal love,
 who we still refuse to accept.

The Lord has sent me. Blind, barren. Sleeping in
############
Strong son of god, immortal love,
 who weeps for parents whose loves were broken, I am your son."

This may be uttered
############
Strong son of god, immortal love,
 who we know as the Aett, leads a godly life.

Deathwards and forwards

############


In [12]:
tokenizer("hello there \n how are you")
tokenizer("hello there how are you")
tokenizer("hello there, how are you")

{'input_ids': [73491, 120, 433, 37, 304]}

{'input_ids': [73491, 120, 433, 37, 304]}

{'input_ids': [73491, 120, 2, 433, 37, 304]}

In [21]:
vocab['idx2sym'][:50]

['<eos>',
 'the',
 ',',
 '.',
 'of',
 'and',
 'to',
 'in',
 'a',
 '=',
 '"',
 'was',
 '@-@',
 'The',
 "'s",
 'on',
 'that',
 'for',
 'as',
 'with',
 'by',
 ')',
 '(',
 'is',
 '<unk>',
 'his',
 'from',
 'at',
 'were',
 'it',
 'he',
 'an',
 'had',
 'In',
 'which',
 'be',
 'but',
 'are',
 'not',
 ';',
 'first',
 'their',
 'â\x80\x93',
 ':',
 'also',
 'her',
 'its',
 'or',
 'have',
 'one']

In [17]:
len(vocab['idx2sym']),
len(cmu)

(267735,)

123455

In [18]:
'The' in vocab['idx2sym']

True

In [19]:
vocab['idx2sym'].index('The')

13

In [23]:
vocab_valid = [x for x in vocab['idx2sym'] if x.lower() in cmu]

In [24]:
len(vocab_valid)

103932